In [ ]:
# Load necessary libraries
import sys
import os
import anndata as ad
import pandas as pd
import scanpy as sc
import seaborn as sns; sns.set(color_codes=True)
import scimap as sm

import squidpy as sq


In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
sc.logging.print_header()

In [ ]:
import glob
inFiles = glob.glob('McmCells/*CellsNID.csv')
inFiles2 = glob.glob('McmCells/*CellsID.csv')

inFiles

In [ ]:
detDF = pd.read_csv('McmCells/detection_measurements_post_threshold_Cells_1116.tsv',sep='\t')

detDF

In [ ]:
detDF

In [ ]:
cellDF = detDF[detDF.Class=='Cells']

for im in cellDF.Image.unique():
    print(cellDF[cellDF.Image==im].ID.values[:10])


In [ ]:
#cellDF.ImageID.isna().sum()

In [ ]:
imageIDMapQP = {'10202021_BaharehNP_2997-ADCortex_EDTA.qptiff - resolution #1':'2997',
       '11162021_Bahareh-3155_EDTA.qptiff - resolution #1':'3155',
       '10122021_BaharehNP_ADCortex_EDTA.qptiff - resolution #1':'3196',
       '12032021-Bahareh_3280_Alz.qptiff - resolution #1':'3280',
       '10122021_BaharehNP_1-CtlCortex1796_EDTA_MKT6.qptiff - resolution #1':'1796',
       '12252021_Bahareh-3628-Cntr.qptiff - resolution #1':'3628',
       '11092021_BaharehNP_3026CtlCorte.qptiff - resolution #1':'3026',
       '10222021_BaharehNP_1873_CtrlCortex.qptiff - resolution #1':'1873'}
imageTypeMapQP = {'10202021_BaharehNP_2997-ADCortex_EDTA.qptiff - resolution #1':'AD',
       '11162021_Bahareh-3155_EDTA.qptiff - resolution #1':'AD',
       '10122021_BaharehNP_ADCortex_EDTA.qptiff - resolution #1':'AD',
       '12032021-Bahareh_3280_Alz.qptiff - resolution #1':'AD',
       '10122021_BaharehNP_1-CtlCortex1796_EDTA_MKT6.qptiff - resolution #1':'Ctl',
       '12252021_Bahareh-3628-Cntr.qptiff - resolution #1':'Ctl',
       '11092021_BaharehNP_3026CtlCorte.qptiff - resolution #1':'Ctl',
       '10222021_BaharehNP_1873_CtrlCortex.qptiff - resolution #1':'Ctl'}

cellDF.loc[:,'ImageID'] = cellDF.Image.map(imageIDMapQP)
cellDF = cellDF[cellDF.ImageID.isin(['3026','3155'])]
cellDF.ID = cellDF.ID.astype(int)

In [ ]:
#gliaDF=gliaDF.dropna(axis=1)
cellDF['Distance to annotation PlaquesA µm']

In [ ]:
cellDF.columns

In [ ]:
cellDF.loc[:,'ImageID_CellID'] = cellDF.ImageID +'_'+ cellDF.ID.astype(str)
cellDF

In [ ]:
cellDF[cellDF.ImageID_CellID=='3155_110086']

In [ ]:
ID_ParentDict = pd.Series(cellDF.Parent.values,index=cellDF.ImageID_CellID).to_dict()


In [ ]:
gID_Dict = pd.Series(cellDF.gliaID.values,index=cellDF.ImageID_CellID).to_dict()
nID_Dict = pd.Series(cellDF.neuronID.values,index=cellDF.ImageID_CellID).to_dict()
oID_Dict = pd.Series(cellDF.oligoID.values,index=cellDF.ImageID_CellID).to_dict()


In [ ]:
inFiles

In [ ]:
imageIDMap = {x:x.split('_')[-2] for x in inFiles}
imageIDMap

In [ ]:
imageTypeMap = {}
for x in inFiles:
    if ('AD' in x) or ('3155' in x) or ('Alz' in x):
        imageTypeMap[x] = 'AD'
    else:
        imageTypeMap[x] = 'Ctl'

In [ ]:
imageTypeMap

In [ ]:
reg001 = []
aMap = {'Olig-2':'Olig2','Claudin-5':'Claudin5'}
protNames =  []
for i,iF in enumerate(inFiles):
    print(iF)
    inD = pd.read_csv(iF, sep=',', header=0, index_col=0).rename(columns=aMap,inplace=False)
    inD.columns = ['Nucleus_'+col for col in inD.columns]
    reg001.append(inD)
    reg001[i].loc[:,'Image'] = iF

reg002 = []
aMap = {'Olig-2':'Olig2','Claudin-5':'Claudin5'}

for i,iF in enumerate(inFiles2):
    print(iF)
    inD = pd.read_csv(iF, sep=',', header=0, index_col=0).rename(columns=aMap,inplace=False)
    
    reg002.append(inD.loc[reg001[i].index])
    #reg002[i].loc[:,'Image'] = iF


In [ ]:
reg001[0].shape, reg002[0].shape, reg001[1].shape, reg002[1].shape, 

In [ ]:
reg001[1]

In [ ]:
i1 = pd.concat(reg001)
i2 = pd.concat(reg002)#.loc[i1.index]
combinedDF = pd.concat([i1,i2],axis=1)
#combinedDF = combinedDF[combinedDF['Centroid Y µm'] >= 4800]
combinedDF.reset_index(inplace=True)


combinedDF['ImageID'] = combinedDF.Image.map(imageIDMap)
combinedDF['ImageType'] = combinedDF.Image.map(imageTypeMap)
combinedDF['ImageID_CellID'] = combinedDF.ImageID +'_'+combinedDF.CellID.astype(str)
combinedDF['Parent'] = combinedDF.ImageID_CellID.map(ID_ParentDict)
combinedDF['gliaID'] = combinedDF.ImageID_CellID.map(gID_Dict)
combinedDF['neuronID'] = combinedDF.ImageID_CellID.map(nID_Dict)
combinedDF['oligoID'] = combinedDF.ImageID_CellID.map(oID_Dict)

combinedDF.head()


In [ ]:
combinedDF.shape, i1.shape, i2.shape

In [ ]:
combinedDF.Parent.value_counts()
combinedDF = combinedDF[combinedDF.Parent != 'Image']
junkCells = pd.read_csv('McmCells/to_drop.csv')
combinedDF = combinedDF[~combinedDF.ImageID_CellID.isin(junkCells.ImageID_CellID.values)]

In [ ]:
pd.read_csv('McmCells/to_drop.csv')


In [ ]:
sorted(combinedDF.columns)

In [ ]:
keepCols = ['GFAP','IBA1','Nucleus_Olig2','Nucleus_NeuN']#,'Nucleus_CollagenIV']

protDF = combinedDF[keepCols]
protDF.head()

In [ ]:
protDF.isna().sum(axis=0)

In [ ]:
newNames = {col:col.split('_')[-1] for col in protDF.columns}
protDF.rename(newNames, inplace=True, axis=1)
protDF.head()

In [ ]:
protDF.columns

In [ ]:
combinedDF

In [ ]:
#spatialCols = ['Centroid X µm','Centroid Y µm','Cell: Area µm^2']
#spatialNames  = {'Centroid X µm':'spatial_X','Centroid Y µm': 'spatial_Y','Cell: Area µm^2': 'Area'}#,'Image':'ImageID'}
spatialCols = ['X_centroid','Y_centroid','Area','ImageID','ImageType','Parent','ImageID_CellID','gliaID','neuronID','oligoID']
spatialNames  = {'X_centroid':'spatial_X','Y_centroid': 'spatial_Y','Area': 'Area'}#,'Image':'ImageID'}

spatDF = combinedDF[spatialCols]
spatDF.rename(columns=spatialNames, inplace=True)
#spatDF['ImageID'] = spatDF.spatial_Y >= 4000
#spatDF['ImageID'] = spatDF['ImageID'].astype('int')
spatDF

In [ ]:
# Load data
# combine the data and metadata file to generate the AnnData object
adata = ad.AnnData (protDF)
adata.obs = spatDF

adata.obs['oldIndex'] = adata.obs.index
adata.obs.reset_index(inplace=True, drop=True)

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages
base = importr('CELESTA')
import numpy as np
# Defining the R script and loading the instance in Python
r = robjects.r
r['source']('WriteNormExp.R')

In [ ]:
adata.obs.ImageID.unique()
adata.obs

In [ ]:
imIDS = adata.obs.ImageID.unique()
for imID in imIDS:
    subD =adata[adata.obs.ImageID==imID]
    protDF = pd.DataFrame(subD.X, columns=subD.var_names, index=subD.obs.index)
    #outDF = pd.concat([protDF[['NeuN','GFAP','Olig2','IBA1','CollagenIV']],subD.obs],axis=1)
    outDF = pd.concat([protDF[['NeuN','GFAP','Olig2','IBA1']],subD.obs],axis=1)
    outDF['oldIndex'] = outDF.index.values
    outDF.reset_index(inplace=True,drop=True)
    outDF.to_csv(f'{imID}_corr.csv')

    # Loading the function we have defined in R.
    writeNorm = robjects.globalenv['WriteNormExp']
    writeNorm(imID,'phenotype_Celesta.csv', f'{imID}_corr.csv','Other',f'{imID}Prob.csv')


In [ ]:
inF1 = pd.concat([pd.read_csv(f'{imIDS[0]}Prob.csv'),pd.read_csv(f'{imIDS[1]}Prob.csv')])

In [ ]:
adata.X = inF1[adata.var_names].values

In [ ]:
#sc.pp.log1p(adata)
#adataScaled=sc.pp.scale(adata, max_value=10, copy=True)
#sc.pp.combat(adata, key='ImageID', inplace=True)
#adataScaled=sc.pp.scale(adata, max_value=10, copy=True)

#sc.pp.regress_out(adataScaled,keys='spatial_Y')
#adataScaled = rescale(adata,imageid = 'ImageID', failed_markers=['CD68'], save_fig=True)

#sm.pp.rescale(adata,method='by_imageID', imageid='ImageID',)

In [ ]:
adata.X

In [ ]:

pheno = pd.read_csv('sm_phenotype.csv', index_col=None, header=0)
pheno

In [ ]:
pd.DataFrame(adata.X, columns=adata.var_names)

In [ ]:
np.median(adata.X[:,0]), np.median(adata.X[:,1]), np.median(adata.X[:,2]), np.median(adata.X[:,3])


In [ ]:
adataScaled = sm.tl.phenotype_cells(adata, phenotype= pheno, gate=0.5)


In [ ]:
100*adataScaled.obs.groupby(['ImageID','Parent']).phenotype.value_counts()/adataScaled.obs.groupby(['ImageID','Parent']).size()

In [ ]:
adataScaled.obs.groupby(['ImageID','Parent']).phenotype.value_counts()

In [ ]:
adataScaled.obs.groupby(['ImageID']).size()

In [ ]:
adataScaled.obs.groupby(['ImageID']).phenotype.value_counts()

In [ ]:
adataScaled.obs.groupby(['ImageID']).Parent.value_counts(), adataScaled.obs.groupby(['ImageID']).size()

In [ ]:
adataScaled.obs[adataScaled.obs.ImageID=='3155'].groupby('Parent').size()

In [ ]:
adataScaled.obs[adataScaled.obs.ImageID=='3155'].groupby('Parent').size()

In [ ]:
adataScaled.obs.reset_index(inplace=True, drop=True)
adataScaled.obs

In [ ]:
subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155'])]
subDF

In [ ]:
(subDF.neuronID != -1).sum(), subDF[subDF.phenotype=='Neurons'].shape[0]

In [ ]:
(subDF.gliaID != -1).sum(), subDF[subDF.phenotype=='Microglia'].shape[0]

In [ ]:
(subDF.oligoID != -1).sum(), subDF[subDF.phenotype=='Oligodendrocytes'].shape[0]

In [ ]:
(subDF[subDF.neuronID != -1].phenotype=='Neurons').sum()/subDF[subDF.neuronID != -1].shape[0]

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
cNames = ['Gate','Neurons_AD','Neurons_Ctl','Neurons_Combined',
          'OligoD_AD','OligoD_Ctl','OligoD_Combined','Other_AD','Other_Ctl','Other_Combined']
pDF = pd.DataFrame(index=[x for x in range(1,20)], columns=cNames)
rDF = pd.DataFrame(index=[x for x in range(1,20)], columns=cNames)
aDF = pd.DataFrame(index=[x for x in range(1,20)], columns=cNames)
for gateX in range(1,20):
    print("Using gate: ",round(gateX*0.05,2))
    adataScaled = sm.tl.phenotype_cells(adata, phenotype= pheno, gate=round(gateX*0.05,2))
    pDF.loc[gateX, 'Gate'] = round(gateX*0.05,2)
    rDF.loc[gateX, 'Gate'] = round(gateX*0.05,2)
    aDF.loc[gateX, 'Gate'] = round(gateX*0.05,2)
    subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155'])]

    pDF.loc[gateX, 'Neurons_AD'] = precision_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    rDF.loc[gateX, 'Neurons_AD'] = recall_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    aDF.loc[gateX, 'Neurons_AD'] = accuracy_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))

    pDF.loc[gateX, 'OligoD_AD'] = precision_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    rDF.loc[gateX, 'OligoD_AD'] = recall_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    aDF.loc[gateX, 'OligoD_AD'] = accuracy_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))

    pDF.loc[gateX, 'Other_AD'] = precision_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    rDF.loc[gateX, 'Other_AD'] = recall_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    aDF.loc[gateX, 'Other_AD'] = accuracy_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
                                                
    subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3026'])]


    pDF.loc[gateX, 'Neurons_Ctl'] = precision_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    rDF.loc[gateX, 'Neurons_Ctl'] = recall_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    aDF.loc[gateX, 'Neurons_Ctl'] = accuracy_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))

    pDF.loc[gateX, 'OligoD_Ctl'] = precision_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    rDF.loc[gateX, 'OligoD_Ctl'] = recall_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    aDF.loc[gateX, 'OligoD_Ctl'] = accuracy_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))

    pDF.loc[gateX, 'Other_Ctl'] = precision_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    rDF.loc[gateX, 'Other_Ctl'] = recall_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    aDF.loc[gateX, 'Other_Ctl'] = accuracy_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))

    subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155','3026'])]


    pDF.loc[gateX, 'Neurons_Combined'] = precision_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    rDF.loc[gateX, 'Neurons_Combined'] = recall_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))
    aDF.loc[gateX, 'Neurons_Combined'] = accuracy_score((subDF.neuronID != -1), (subDF.phenotype=='Neurons'))

    pDF.loc[gateX, 'OligoD_Combined'] = precision_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    rDF.loc[gateX, 'OligoD_Combined'] = recall_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
    aDF.loc[gateX, 'OligoD_Combined'] = accuracy_score((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'))
                                                 
    pDF.loc[gateX, 'Other_Combined'] = precision_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    rDF.loc[gateX, 'Other_Combined'] = recall_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))
    aDF.loc[gateX, 'Other_Combined'] = accuracy_score((subDF.oligoID == -1) & (subDF.neuronID == -1),
                                                  ~subDF.phenotype.isin(['Neurons','Oligodendrocytes']))

In [ ]:
import matplotlib.pyplot as plt
adataScaled = sm.tl.phenotype_cells(adata, phenotype= pheno, gate=0.5)


In [ ]:
subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155','3026'])]

subDF['isNeuron'] = subDF.neuronID != -1
subDF['isOligo'] = subDF.oligoID != -1
subDF['isOther'] = (subDF.oligoID == -1) & (subDF.neuronID == -1)

subDF['isPredNeuron'] = subDF.phenotype=='Neurons'
subDF['isPredOligo'] = subDF.phenotype=='Oligodendrocytes'
subDF['isPredOther'] = ~subDF.phenotype.isin(['Neurons','Oligodendrocytes'])
#y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).astype('category').cat.codes.values
#y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category').cat.codes.values
y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).map({'isNeuron':0,'isOligo':1,'isOther':2 }).values
y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).map({'isPredNeuron':0,'isPredOligo':1,'isPredOther':2 }).values

fig, ax=plt.subplots(1,1,figsize=(5,5))
sns.heatmap(confusion_matrix(y_true,y_pred,normalize='true' )*100,
            cmap='viridis', annot=True, fmt='.2f',ax=ax)


In [ ]:
subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155'])]
subDF['isNeuron'] = subDF.neuronID != -1
subDF['isOligo'] = subDF.oligoID != -1
subDF['isOther'] = (subDF.oligoID == -1) & (subDF.neuronID == -1)

subDF['isPredNeuron'] = subDF.phenotype=='Neurons'
subDF['isPredOligo'] = subDF.phenotype=='Oligodendrocytes'
subDF['isPredOther'] = ~subDF.phenotype.isin(['Neurons','Oligodendrocytes'])

#y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).astype('category').cat.codes.values
#y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category').cat.codes.values
y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).map({'isNeuron':0,'isOligo':1,'isOther':2 }).values
y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).map({'isPredNeuron':0,'isPredOligo':1,'isPredOther':2 }).values

fig, ax=plt.subplots(1,1,figsize=(5,5))
sns.heatmap(confusion_matrix(y_true,y_pred,normalize='true' )*100,
            cmap='viridis', annot=True, fmt='.2f',ax=ax)


In [ ]:
import matplotlib as mpl
#[255*x for x in mpl.colors.hex2color('#00dbf2')]
cMP =mpl.colors.LinearSegmentedColormap.from_list("", [(66, 46, 51),(236, 252, 160)])
cMP = mpl.colors.LinearSegmentedColormap.from_list("", ['#422e33','#5b535f','#75ab9d','#89d188','#FDE725'])
cMP

In [ ]:
subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155'])]
subDF['isNeuron'] = subDF.neuronID != -1
subDF['isOligo'] = subDF.oligoID != -1
subDF['isOther'] = (subDF.oligoID == -1) & (subDF.neuronID == -1)

subDF['isPredNeuron'] = subDF.phenotype=='Neurons'
subDF['isPredOligo'] = subDF.phenotype=='Oligodendrocytes'
subDF['isPredOther'] = ~subDF.phenotype.isin(['Neurons','Oligodendrocytes'])

#y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).astype('category').cat.codes.values
#y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category').cat.codes.values
y_true = subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).map({'isNeuron':0,'isOligo':1,'isOther':2 }).values
y_pred = subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).map({'isPredNeuron':0,'isPredOligo':1,'isPredOther':2 }).values

fig, ax=plt.subplots(1,1,figsize=(4,4))
result = confusion_matrix(y_true,y_pred,normalize='true' )*100
labels = (np.asarray(["{0:.2f}%".format(value) for value in result.flatten()])).reshape(3, 3)

sns.heatmap(result, cmap=cMP,vmax=100,vmin=0, annot=labels, fmt='', ax=ax,cbar=False)
ax.set_xticklabels(['Neurons','OligoD','Rest'])
ax.set_ylabel('True label')
ax.set_yticklabels(['Neurons','OligoD','Rest'])
ax.set_xlabel('Predicted label')

In [ ]:
result

In [ ]:
(np.asarray(["{0:.2f}%".format(value) for value in result.flatten()])).reshape(3, 3)

In [ ]:
subDF = adataScaled.obs.copy()
subDF['isNeuron'] = subDF.neuronID != -1
subDF['isOligo'] = subDF.oligoID != -1
subDF['isOther'] = (subDF.oligoID == -1) & (subDF.neuronID == -1)

subDF['isPredNeuron'] = subDF.phenotype=='Neurons'
subDF['isPredOligo'] = subDF.phenotype=='Oligodendrocytes'
subDF['isPredOther'] = ~subDF.phenotype.isin(['Neurons','Oligodendrocytes'])
subDF1 = subDF[subDF.ImageID.isin(['3155'])]
print(accuracy_score(subDF1['isNeuron'], subDF1['isPredNeuron']), accuracy_score(subDF1['isOligo'], subDF1['isPredOligo']),  accuracy_score(subDF1['isOther'], subDF1['isPredOther']))
subDF2 = subDF[subDF.ImageID.isin(['3026'])]
print(accuracy_score(subDF2['isNeuron'], subDF2['isPredNeuron']), accuracy_score(subDF2['isOligo'], subDF2['isPredOligo']),  accuracy_score(subDF2['isOther'], subDF2['isPredOther']))
subDF2 = subDF.copy()
print(accuracy_score(subDF2['isNeuron'], subDF2['isPredNeuron']), accuracy_score(subDF2['isOligo'], subDF2['isPredOligo']),  accuracy_score(subDF2['isOther'], subDF2['isPredOther']))

In [ ]:
aDF[aDF.Gate==0.5][['Neurons_AD','OligoD_AD','Other_AD']].values[0]*100

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(4,5))
sns.barplot(y=aDF[aDF.Gate==0.5][['Neurons_AD','OligoD_AD','Other_AD']].values[0]*100,
            x=['Neurons','OligoD','Rest'], ax=ax, palette=['#4CAF50','#FF0055', '#C8C8C8'])
ax.set_ylabel("Accuracy (%)")
ax.set_ylim(50,100)


In [ ]:
fig,ax=plt.subplots(1,1,figsize=(4,5))
sns.barplot(y=aDF[aDF.Gate==0.5][['Neurons_Ctl','OligoD_Ctl','Other_Ctl']].values[0]*100,
            x=['Neurons','OligoD','Rest'], ax=ax, palette=['#4CAF50','#FF0055', '#C8C8C8'])
ax.set_ylabel("Accuracy (%)")
ax.set_ylim(50,100)
print(aDF[aDF.Gate==0.5][['Neurons_Ctl','OligoD_Ctl','Other_Ctl']].values[0]*100)

In [ ]:
subDF.isPredOligo.value_counts(), subDF.isOligo.value_counts()

In [ ]:
subDF.phenotype.value_counts()

In [ ]:
subDF.phenotype

In [ ]:
subDF

In [ ]:
subDF[['isNeuron','isOligo','isOther']].idxmax(axis=1).astype('category').cat.codes.values

In [ ]:
subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category').cat.codes.values

In [ ]:
subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category')

In [ ]:
subDF[['isPredNeuron','isPredOligo','isPredOther']].idxmax(axis=1).astype('category')

In [ ]:
y_true[0], y_pred

In [ ]:
subDF = adataScaled.obs[adataScaled.obs.ImageID.isin(['3155'])]

fig, ax=plt.subplots(2,1,figsize=(3,5))
sns.heatmap(confusion_matrix((subDF.neuronID != -1), (subDF.phenotype=='Neurons'),normalize='true')*100,
            cmap='viridis', annot=True, fmt='.2f',ax=ax[0])
ax[0].set_xticklabels([])
ax[0].set_ylabel('True')
ax[0].set_title('Neurons')

sns.heatmap(confusion_matrix((subDF.oligoID != -1), (subDF.phenotype=='Oligodendrocytes'),normalize='true')*100,
            cmap='viridis', annot=True, fmt='.2f',ax=ax[1])
ax[1].set_ylabel('True')
ax[1].set_xlabel('Predicted')
ax[1].set_title('Oligodendrocytes')

In [ ]:
#sns.scatterplot(x=pDF.Neurons_AD,y=aDF.Neurons_AD, hue=pDF.Gate)
#sns.scatterplot(x=pDF.OligoD_Combined,y=aDF.OligoD_Combined, hue=pDF.Gate)

In [ ]:
aDF.plot(x='Gate',y=['Neurons_AD','Neurons_Ctl','Neurons_Combined'])
aDF.plot(x='Gate',y=['OligoD_AD','OligoD_Ctl','OligoD_Combined'])

In [ ]:
aDF.plot(x='Gate',y=['Neurons_AD','Neurons_Ctl','Neurons_Combined'])
aDF.plot(x='Gate',y=['OligoD_AD','OligoD_Ctl','OligoD_Combined'])

pDF.plot(x='Gate',y=['Neurons_AD','Neurons_Ctl','Neurons_Combined'])
pDF.plot(x='Gate',y=['OligoD_AD','OligoD_Ctl','OligoD_Combined'])

rDF.plot(x='Gate',y=['Neurons_AD','Neurons_Ctl','Neurons_Combined'])
rDF.plot(x='Gate',y=['OligoD_AD','OligoD_Ctl','OligoD_Combined'])

In [ ]:
pDF[['Neurons_Combined','OligoD_Combined']].mean(axis=1).idxmax(), rDF[['Neurons_Combined','OligoD_Combined']].median(axis=1).idxmax(), aDF[['Neurons_Combined','OligoD_Combined']].mean(axis=1).idxmax()

In [ ]:
aDF[['Neurons_Combined','OligoD_Combined']].mean(axis=1).max()

In [ ]:
pd.concat([pDF[['Gate','Neurons_Combined','OligoD_Combined']],
           rDF[['Neurons_Combined','OligoD_Combined']],
           aDF[['Neurons_Combined','OligoD_Combined']]],axis=1)

In [ ]:
adataScaled = sm.tl.phenotype_cells(adata, phenotype= pheno, gate=0.5)


In [ ]:
adataScaled.obs.phenotype = adataScaled.obs.phenotype.map({'Neurons':'Neurons',
                                                           'Astrocytes':'Astrocytes',
                                                           'Microglia':'Microglia',
                                                           'Oligodendrocytes':'Oligodendrocytes',
                                                           'Unknown':'Other'})

In [ ]:
adataScaled.obs.phenotype.value_counts()

In [ ]:
sc.pl.palettes.godsnot_102[:8]

In [ ]:
cL2=['#FFFF00', '#00DBF2', '#33FF33','#ff00ff', '#FF0055', '#EB984E','#3366FF']
cL=['#FDD835', '#00DBF2', '#4CAF50','#FF0055', '#3366FF']
#cL = ['#FFFF00', '#00DBF2', '#33FF33', '#FF0055', '#ff00ff','#EB984E','#3366FF']
cMapDict = {sorted(adataScaled.obs.phenotype.unique())[i]: cL[i] for i in range(len(adataScaled.obs.phenotype.unique()))}
cMapDict

In [ ]:
adataScaled.obsm['X_spatial'] = adataScaled.obs[['spatial_X','spatial_Y']].values
adataScaled.obsm['X_spatial'][:,1] = -1*adataScaled.obsm['X_spatial'][:,1]
#adataScaled.write('../vitesscePy/QuPathDemo/1796allProt.h5ad')

import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(12,8))
ss = adataScaled[adataScaled.obs.ImageID=='3155']
#ss.obs.phenotype=ss.obs.phenotype.astype('category')

sc.pl.scatter(ss, basis='spatial',color=['phenotype'],ax=ax,size=9,title='',save='AD_Class',
              alpha=0.9, palette=cMapDict.values())
#save='AD_Class.png',
#fig,ax=plt.subplots(1,1,figsize=(12,8))
#ss = sc.pp.subsample(adataScaled[adataScaled.obs.ImageID=='3155'],fraction=1.0, copy=True)
#sc.pl.scatter(ss, basis='spatial',color=['leiden'],ax=ax,size=10,title='',save='ADLx2.png',alpha=0.9,palette=sc.pl.palettes.godsnot_102)

In [ ]:
adataScaled.obsm['X_spatial'] = adataScaled.obs[['spatial_X','spatial_Y']].values
adataScaled.obsm['X_spatial'][:,1] = -1*adataScaled.obsm['X_spatial'][:,1]
#adataScaled.write('../vitesscePy/QuPathDemo/1796allProt.h5ad')

import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(12,8))
ss = adataScaled[adataScaled.obs.ImageID=='3026']
#ss.obs.phenotype=ss.obs.phenotype.astype('category')

sc.pl.scatter(ss, basis='spatial',color=['phenotype'],ax=ax,size=10,title='',save='Ctl_Class.png',alpha=0.9, palette=cL)
#fig,ax=plt.subplots(1,1,figsize=(12,8))
#ss = sc.pp.subsample(adataScaled[adataScaled.obs.ImageID=='3155'],fraction=1.0, copy=True)
#sc.pl.scatter(ss, basis='spatial',color=['leiden'],ax=ax,size=10,title='',save='ADLx2.png',alpha=0.9,palette=sc.pl.palettes.godsnot_102)

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,8))
sns.set_style('white')
ss = adataScaled[adataScaled.obs.ImageID=='3155']

sns.scatterplot(data=ss.obs,x='spatial_X',y='spatial_Y',hue='phenotype', 
                palette=cMapDict, s=2, alpha=0.9)
ax.invert_yaxis()
plt.axis('off')
ax.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/AD_Class.png', dpi=300)

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(9,8))
sns.set_style('white')
ss = adataScaled[adataScaled.obs.ImageID=='3026']

sns.scatterplot(data=ss.obs,x='spatial_X',y='spatial_Y',hue='phenotype', 
                palette=cMapDict, s=2)
ax.invert_yaxis()
plt.axis('off')
ax.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/Ctl_Class.png', dpi=300)

In [ ]:
ss = adataScaled[adataScaled.obs.ImageID=='3155']

100*(ss.obs.groupby(['Parent','phenotype']).size())/ss.obs.groupby(['Parent']).size()

In [ ]:
ss

In [ ]:
75+4.2+3.3+8.4+9.1

In [ ]:
#!pip install descartes

In [ ]:
import alphashape
from descartes import PolygonPatch

ssN = ss.obs[(ss.obs.Parent=='Grey Matter') & (ss.obs.spatial_X)<5000]
print(ssN.shape)
alp = alphashape.alphashape(ssN[['spatial_X','spatial_Y']].values,1)

In [ ]:
alp.geoms

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,8))
sns.set_style('white')
sns.scatterplot(data=ss.obs,x='spatial_X',y='spatial_Y',hue='Parent', 
                palette=['tab:blue','tab:pink'], s=1.5)
ax.add_patch(PolygonPatch(alp, alpha=0.5))
ax.invert_yaxis()
plt.axis('off')
ax.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/AD_Class.png', dpi=300)

In [ ]:


import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(12,8))
ss = adataScaled[adataScaled.obs.ImageID=='3155']
#ss.obs.phenotype=ss.obs.phenotype.astype('category')

sc.pl.scatter(ss, basis='spatial',color=['Parent'],ax=ax,size=9,title='',
              alpha=0.9, palette=['tab:blue','tab:pink'])

plt.axis('off')
plt.savefig('finalFiguresv4/AD_Region.png', dpi=300)
#save='AD_Class.png',
#fig,ax=plt.subplots(1,1,figsize=(12,8))
#ss = sc.pp.subsample(adataScaled[adataScaled.obs.ImageID=='3155'],fraction=1.0, copy=True)
#sc.pl.scatter(ss, basis='spatial',color=['leiden'],ax=ax,size=10,title='',save='ADLx2.png',alpha=0.9,palette=sc.pl.palettes.godsnot_102)

In [ ]:
#!pip install alphashape

In [ ]:
sc.pl.matrixplot(adataScaled, groupby='phenotype', figsize=(6,3),var_names=['NeuN','GFAP','Olig2','IBA1'],categories_order = ['Neurons','Astrocytes','Oligodendrocytes','Microglia','Other'],
                 dendrogram=False, use_raw=False, cmap="inferno",standard_scale=None,vmin=-0.0,vmax=0.7,vcenter=0.5, return_fig=False,swap_axes=False, save='_ClassificationCells.png')

In [ ]:
ss.obs.phenotype

In [ ]:
#import rapids_scanpy_funcs as rs


In [ ]:
sc.tl.pca(adataScaled, svd_solver='arpack') # peform PCA
#sc.pl.pca(adataScaled, color=protDF.columns)
#sc.pl.pca(adataScaled, color=['ImageID'])


In [ ]:
sc.pl.pca(sc.pp.subsample(adataScaled,fraction=0.1, copy=True), color=protDF.columns,use_raw=False)

In [ ]:
#sc.pp.neighbors(adataScaled, n_neighbors=30, n_pcs=20) # Computing the neighborhood graph
sc.pp.neighbors(adataScaled, n_neighbors=50, n_pcs=15) # Computing the neighborhood graph


In [ ]:
adataScaled

In [ ]:
sc.tl.umap(adataScaled) # Build a UMAP to visualize the neighbourhood graph


In [ ]:
sc.pl.umap(sc.pp.subsample(adataScaled,fraction=0.5, copy=True), color=protDF.columns, save='xx.png', use_raw=False)


In [ ]:
sc.pl.umap(adataScaled, color=['phenotype'], palette=sc.pl.palettes.vega_10)


sc.pl.umap(adataScaled, color=['Parent'], palette=sc.pl.palettes.godsnot_102[1:3])
sc.pl.umap(adataScaled, color=['ImageType'], palette=sc.pl.palettes.godsnot_102[3:5])
sc.pl.umap(adataScaled, color=['ImageID'], palette=sc.pl.palettes.godsnot_102[5:13])

In [ ]:
sc.pl.umap(adataScaled, color=['ImageID','phenotype','ImageType','Parent'], ncols=2,save='_Cell_seg_Classification.png')

In [ ]:
adataScaled.obs

In [ ]:
adataScaled.obs.phenotype.value_counts(sort=True)

In [ ]:
adataScaled.obs['Annotations']=adataScaled.obs.phenotype.values

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(12,8))
#ss = sc.pp.subsample(adataScaled[adataScaled.obs.ImageID=='3026'],fraction=1.0, copy=True)
ss = adataScaled[adataScaled.obs.ImageID=='3026']
#ss.obs.Annotations=ss.obs.Annotations.astype('category')

sc.pl.scatter(ss, basis='spatial',color=['Annotations'],ax=ax,size=8,title='',save='CtlLxClass.png',alpha=0.9, )
fig,ax=plt.subplots(1,1,figsize=(12,8))
ss = adataScaled[adataScaled.obs.ImageID=='3155']
sc.pl.scatter(ss, basis='spatial',color=['Annotations'],ax=ax,size=8,title='',save='ADLxClass.png',alpha=0.9)

In [ ]:
sc.pl.matrixplot(adataScaled, groupby='phenotype', figsize=(8,3),var_names=['Olig2','NeuN','GFAP','IBA1'],
                 dendrogram=True, use_raw=False, cmap="inferno",standard_scale=None,vmin=0,vmax=0.75,vcenter=0.5, return_fig=False,swap_axes=False, save='_ClusteringCells.png')

In [ ]:
corrDF = protDF.corr('spearman')

In [ ]:
import matplotlib.pyplot as plt
#fig,ax=plt.subplots(1,1,figsize=(20,20))
ax3 = sns.clustermap(corrDF, vmin=0, vmax=1, figsize=(8,8),yticklabels=True, xticklabels=True, cmap='inferno')

In [ ]:
adataScaled.obsm['X_spatial'] = adataScaled.obs[['spatial_X','spatial_Y']].values
adataScaled.obsm['X_spatial'][:,1] = -1*adataScaled.obsm['X_spatial'][:,1]
adataScaled.write('Fig3.h5ad')

In [ ]:
#adataScaled.write_zarr('../vitesscePy/QuPathDemo/1796_subsetP.zarr')

In [ ]:
ad.__version__

In [ ]:
#ad.read_csv(

In [ ]:
adataScaled.obs

In [ ]:
#adataScaled.obs.to_csv('1796_leiden.csv')
adataScaled.obs.ImageID.unique()

In [ ]:
sc.pl.umap(adataScaled, color=['phenotype'], palette=sc.pl.palettes.vega_10, save='_CellsAnnotated_Class.png')

In [ ]:
cMapDict = {sorted(adataScaled.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled.obs.Annotations.unique()))}

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(8,8))
ax.pie(adataScaled.obs.Annotations.value_counts(),explode=[0.05]*len(adataScaled.obs.Annotations.value_counts().unique()),pctdistance=-0.85,startangle=0,
        labels=[f"{ix}({x})" for ix, x in adataScaled.obs.Annotations.value_counts().items()], colors=[cMapDict[c] for c in adataScaled.obs.Annotations.value_counts().index],
      textprops = {"fontsize":14})
#plt.set_cmap('Set2')

#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax.add_artist(centre_circle)
plt.tight_layout()


In [ ]:
adataScaled.obs['Annotations']=adataScaled.obs.phenotype.values
cL = ['#FDD835', '#00DBF2', '#4CAF50', '#FF0055', '#3366FF']
#cL = ['#FFFF00', '#00DBF2', '#33FF33', '#FF0055', '#ff00ff','#EB984E','#3366FF']
cMapDict = {sorted(adataScaled.obs.Annotations.unique())[i]: cL[i] for i in range(len(adataScaled.obs.Annotations.unique()))}

In [ ]:
cL = ['#FDD835', '#00DBF2', '#4CAF50', '#FF0055', '#3366FF']
#cL = ['#FFFF00', '#00DBF2', '#33FF33', '#FF0055', '#ff00ff','#EB984E','#3366FF']
cMapDict = {sorted(adataScaled.obs.Annotations.unique())[i]: cL[i] for i in range(len(adataScaled.obs.Annotations.unique()))}

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(12,6))
adataScaled2 = adataScaled[adataScaled.obs.ImageType=='Ctl'].copy()

#cMapDict = {sorted(adataScaled2.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled2.obs.Annotations.unique()))}
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='Grey Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index
ax[0].pie(count_DF,explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        #labels=[f"{ix}({x})" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
         labels=[f"{x}" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":12})
#plt.set_cmap('Set2')
ax[0].set_title('Control - Grey Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[0].add_artist(centre_circle)
plt.tight_layout()
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='White Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index

ax[1].pie(count_DF.loc[keepOrder],explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        #labels=[f"{ix}({x})" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
          labels=[f"{x}" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":12})
#plt.set_cmap('Set2')
ax[1].set_title('Control - White Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[1].add_artist(centre_circle)
plt.tight_layout()
plt.savefig('finalFiguresv4/Cells_Control_GMWM_CellNum_Class.png',dpi=600)

In [ ]:
adataScaled.obs.Annotations.value_counts()

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(12,6))
adataScaled2 = adataScaled[adataScaled.obs.ImageType=='AD'].copy()

#cMapDict = {sorted(adataScaled2.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled2.obs.Annotations.unique()))}
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='Grey Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index
ax[0].pie(count_DF,explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        #labels=[f"{ix}({x})" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
         labels=[f"{x}" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":12})
#plt.set_cmap('Set2')
ax[0].set_title('AD - Grey Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[0].add_artist(centre_circle)
plt.tight_layout()
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='White Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index

ax[1].pie(count_DF.loc[keepOrder],explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        #labels=[f"{ix}({x})" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
          labels=[f"{x}" for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":12})
#plt.set_cmap('Set2')
ax[1].set_title('AD - White Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[1].add_artist(centre_circle)
plt.tight_layout()
plt.savefig('finalFiguresv4/Cells_AD_GMWM_CellNum_Class.png',dpi=600)

In [ ]:
#adataScaled.obs['Annotations'] =adataScaled.obs.phenotype.values

In [ ]:
adataScaled

In [ ]:
#cMapDict = {sorted(adataScaled.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled.obs.Annotations.unique()))}

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(12,6))
adataScaled2 = adataScaled[adataScaled.obs.ImageType=='AD'].copy()

#cMapDict = {sorted(adataScaled2.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled2.obs.Annotations.unique()))}
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='Grey Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index
ax[0].pie(count_DF,explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[0].set_title('AD - Grey Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[0].add_artist(centre_circle)
plt.tight_layout()
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='White Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index

ax[1].pie(count_DF.loc[keepOrder],explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[1].set_title('AD - White Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[1].add_artist(centre_circle)
plt.tight_layout()
plt.savefig('finalFiguresv4/Cells_AD_GMWM_Percent_Class.pdf',dpi=600)

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(12,6))
adataScaled2 = adataScaled[adataScaled.obs.ImageType=='Ctl'].copy()

#cMapDict = {sorted(adataScaled2.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled2.obs.Annotations.unique()))}
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='Grey Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index
ax[0].pie(count_DF,explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[0].set_title('Ctl - Grey Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[0].add_artist(centre_circle)
plt.tight_layout()
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='White Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index

ax[1].pie(count_DF.loc[keepOrder],explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[1].set_title('Ctl - White Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[1].add_artist(centre_circle)
plt.tight_layout()
plt.savefig('finalFiguresv4/Cells_Ctl_GMWM_Percent_Class.png',dpi=600)

In [ ]:

fig,ax=plt.subplots(1,2,figsize=(16,8))
ss = adataScaled[(adataScaled.obs.ImageID.isin(['3155']))]
sc.pl.scatter(ss, basis='spatial',color=['Annotations'],ax=ax[0],size=10.0,title=im+ss.obs.ImageType.values[0],alpha=0.9, 
                  show=False)
sc.pl.scatter(ss, basis='spatial',color=['Parent'],ax=ax[1],size=10.0,title=im+ss.obs.ImageType.values[0],alpha=0.9, 
                  show=False, legend_loc='right margin')
plt.tight_layout()
plt.savefig('finalFigures/3155AD_Spatial_Cells_class.png', dpi=300)

In [ ]:

fig,ax=plt.subplots(1,2,figsize=(16,8))
ss = adataScaled[(adataScaled.obs.ImageID.isin(['3026']))]
sc.pl.scatter(ss, basis='spatial',color=['Annotations'],ax=ax[0],size=12.0,title=im+ss.obs.ImageType.values[0],alpha=0.9, 
                  show=False)
sc.pl.scatter(ss, basis='spatial',color=['Parent'],ax=ax[1],size=12.0,title=im+ss.obs.ImageType.values[0],alpha=0.9, 
                  show=False, legend_loc='right margin')
plt.tight_layout()
plt.savefig('finalFigures/3026Cells_Spatial_Cells_class.png', dpi=300)

In [ ]:
adataScaled.obs.ImageID.value_counts()

In [ ]:
adataScaled.obs.groupby(['ImageID','Parent','Annotations']).size().to_csv('McmCells/ClassCellStatistics.tsv')

In [ ]:
adataScaled.obs.spatial_X.max()

## Staticstics

In [ ]:
Annot3026 = adataScaled[adataScaled.obs.ImageID.isin(['3026'])]
Annot3026.obs.reset_index(inplace=True, drop=True)
Annot3026 = Annot3026[(16604 < Annot3026.obs.spatial_X) &( Annot3026.obs.spatial_X < 21913) & (8611 < Annot3026.obs.spatial_Y) &( Annot3026.obs.spatial_Y < 10618)]
Annot3026.obs.reset_index(inplace=True, drop=True)
Annot3026.obs.phenotype.value_counts()

In [ ]:
Annot3026.obs

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,4))
sc.pl.scatter(Annot3026, basis='spatial',color=['phenotype'],ax=ax[0],size=14.0,alpha=0.9,
                  show=False, title=Annot3026.obs.ImageID[0]+' '+Annot3026.obs.ImageType[0])
sc.pl.scatter(Annot3026, basis='spatial',color=['Parent'],ax=ax[1],size=14.0,alpha=0.9, 
                  show=False, legend_loc='right margin', title=Annot3026.obs.ImageID[0]+' '+Annot3026.obs.ImageType[0])

In [ ]:
Annot3155 = adataScaled[adataScaled.obs.ImageID.isin(['3155'])]
Annot3155.obs.reset_index(inplace=True, drop=True)
Annot3155 = Annot3155[(5 < Annot3155.obs.spatial_X) &( Annot3155.obs.spatial_X < 5297) & (0 < Annot3155.obs.spatial_Y) &( Annot3155.obs.spatial_Y < 1991)]
Annot3155.obs.reset_index(inplace=True, drop=True)

Annot3155.obs.phenotype.value_counts()

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,4))
sc.pl.scatter(Annot3155, basis='spatial',color=['phenotype'],ax=ax[0],size=14.0,alpha=0.9,
                  show=False, title=Annot3026.obs.ImageID[0]+' '+Annot3026.obs.ImageType[0])
sc.pl.scatter(Annot3155, basis='spatial',color=['Parent'],ax=ax[1],size=14.0,alpha=0.9, 
                  show=False, legend_loc='right margin', title=Annot3155.obs.ImageID[0]+' '+Annot3155.obs.ImageType[0])

In [ ]:
#first is  3028, second is 3155
#countsDF = pd.DataFrame.from_dict({'Manual':[177,182],'Stardist':[469,257],'Ours':[343 ,221]})
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
countsDF = pd.DataFrame.from_dict({'OTSM':[216 ,124],'Manual':[177,182],'Stardist':[244,165]})
countsDF['cellID']=['Ctl-1','AD-1']
countsDF=countsDF.melt(id_vars=['cellID'])
countsDF.columns = ['CellID','Method','Count']
#sns.catplot(countsDF,x='cellID',hue='variable')
sns.__version__

In [ ]:
countsDF.groupby('Method').mean()

In [ ]:
sns.set(font_scale=1.2)
sns.set_style('white')

g = sns.catplot(data=countsDF,x='CellID',y='Count',hue='Method', kind='bar', palette='Set1',aspect=4/5)
g.despine(bottom = False, left = False, right=False, top=False)
g.set_xlabels('')
#plt.tight_layout()
plt.savefig('finalFigures/Classification_Segmentation_Comparison_50um.png',dpi=300)

In [ ]:
sns.set(font_scale=1.2)
sns.set_style('white')
fig,ax=plt.subplots(1,1,figsize=(4,5))
g = sns.catplot(data=countsDF,x='CellID',y='Count',hue='Method', kind='bar',
                palette='Set1',ax=ax, figsize=(4,5))
g.despine(bottom = False, left = False, right=False, top=False)
#plt.tight_layout()
g.set_xlabels('')
plt.savefig('finalFigures/Classification_Segmentation_Comparison_50um.png',dpi=300)

In [ ]:
sns.set(font_scale=1.2)
sns.set_style('white')
fig,ax=plt.subplots(1,1,figsize=(5,6))
#g= sns.catplot(data=countsDF.groupby('Method').mean().sort_values(by='Count').reset_index(),x='Method',
#                y='Count',hue='Method', kind='bar', palette='Set1',)
g = sns.barplot(data=countsDF.groupby('Method').mean().sort_values(by='Count').reset_index(),x='Method',
                y='Count',hue='Method',palette='Set1',dodge=False,)
g.set_ylabel('Avg. number of cells detected')
g.get_legend().remove()
#g.despine(bottom = False, left = False, right=False, top=False)
plt.tight_layout()
plt.savefig('finalFigures/Classification_Segmentation_Comparison_50um_Merged.png',dpi=300)

In [ ]:
sns.set(font_scale=1.2)
sns.set_style('white')
fig,ax=plt.subplots(1,1,figsize=(5,6))
#g= sns.catplot(data=countsDF.groupby('Method').mean().sort_values(by='Count').reset_index(),x='Method',
#                y='Count',hue='Method', kind='bar', palette='Set1',)
g = sns.barplot(data=countsDF.groupby('Method').sum().sort_values(by='Count').reset_index(),x='Method',
                y='Count',hue='Method',palette='Set1',dodge=False,)
g.set_ylabel('Total number of cells detected')
g.get_legend().remove()
#g.despine(bottom = False, left = False, right=False, top=False)
plt.tight_layout()
plt.savefig('figuresSVG/Fig6i_total.svg')


In [ ]:
#!pip install seaborn==0.12.0
sns.__version__

In [ ]:
sns.set(font_scale=1.2)
sns.set_style('white')
fig,ax=plt.subplots(1,1,figsize=(5,6))
#g= sns.catplot(data=countsDF.groupby('Method').mean().sort_values(by='Count').reset_index(),x='Method',
#                y='Count',hue='Method', kind='bar', palette='Set1',)
g = sns.barplot(data=countsDF,#.groupby('Method').mean().sort_values(by='Count').reset_index(),
                x='Method',capsize=0.2,errorbar='sd',
                y='Count',hue='Method',palette='Set1',dodge=False,)
g.set_ylabel('Avg. number of cells detected')
g.get_legend().remove()
#g.despine(bottom = False, left = False, right=False, top=False)
plt.tight_layout()
plt.savefig('figuresSVG/Fig6i_avg_errorbar.svg')
plt.show()

In [ ]:
g

In [ ]:
countsDF

In [ ]:
nMap = {'Microglia': 'Microglia',
 'Neurons': 'Neurons',
 'Oligodendrocytes': 'Oligodendrocytes',
 'Astrocytes': 'Astrocytes',
'Unknown':'Other'}

In [ ]:
sns.set_style("whitegrid", {'axes.grid' : False})
adataScaled.obs['Annotations']=adataScaled.obs.phenotype.map(nMap)
cL = ['#FDD835', '#00DBF2', '#4CAF50', '#FF0055', '#3366FF']
#cL = ['#FFFF00', '#00DBF2', '#33FF33', '#FF0055', '#ff00ff','#EB984E','#3366FF']
cMapDict = {sorted(adataScaled.obs.Annotations.unique())[i]: cL[i] for i in range(len(adataScaled.obs.Annotations.unique()))}

In [ ]:
cMapDict={'Astrocytes': '#FDD835',
 'Microglia': '#00DBF2',
 'Neurons': '#4CAF50',
 'Oligodendrocytes': '#FF0055',
 'Other': '#3366FF'}

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(12,6))
adataScaled2 = adataScaled[adataScaled.obs.ImageType=='AD'].copy()

#cMapDict = {sorted(adataScaled2.obs.Annotations.unique())[i]: adataScaled.uns['Annotations_colors'][i] for i in range(len(adataScaled2.obs.Annotations.unique()))}
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='Grey Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index
ax[0].pie(count_DF,explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[0].set_title('AD - Grey Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[0].add_artist(centre_circle)
plt.tight_layout()
count_DF  = adataScaled2.obs[adataScaled2.obs.Parent=='White Matter'].sample(frac=1).Annotations.value_counts()
keepOrder = count_DF.index

ax[1].pie(count_DF.loc[keepOrder],explode=[0.05]*len(adataScaled2.obs.Annotations.unique()),pctdistance=-0.85,startangle=0,
        labels=["{:.1f}%".format(100.*x/count_DF.sum()) for ix, x in count_DF.items()], colors=[cMapDict[c] for c in keepOrder],
      textprops = {"fontsize":24})
#plt.set_cmap('Set2')
ax[1].set_title('AD - White Matter')
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
#fig = plt.gcf()
ax[1].add_artist(centre_circle)
plt.tight_layout()
plt.savefig('finalFiguresv4/Cells_AD_GMWM_Percent_Class.pdf',dpi=600)

In [ ]:
adataScaled2 = adataScaled[adataScaled.obs.ImageID=='3026'].copy()

df2 = pd.DataFrame(adataScaled2.obs.groupby('Parent').phenotype.value_counts(normalize=True)*100).round(1)
df2.columns = ['percentage']

df2 = df2.reset_index(0).reset_index()
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

df2['Region'] = df2.Parent.map(nameMap)

#df2['phenotype'] = df2.phenotype.map(nMap)

df2

In [ ]:
plt.rc('axes',edgecolor='k')


In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(4,4))
sns.barplot(data=df2, x='Region', y ='percentage',hue='phenotype',orient='v',hue_order=['Neurons','Oligodendrocytes',
                                                                                        'Astrocytes','Microglia','Other'],
              palette=cMapDict)#['tab:blue','tab:pink'])
ax.set_ylabel('% of cells')
ax.set_xlabel('')
ax.set_yticklabels(['0%','10%','20%','30%','40%','50%','60%','70%'])

plt.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/3026_Percentages.png',dpi=300)
#ax.legend_.set_title('')

In [ ]:
adataScaled2 = adataScaled[adataScaled.obs.ImageID=='3155'].copy()

df2 = pd.DataFrame(adataScaled2.obs.groupby('Parent').phenotype.value_counts(normalize=True)*100).round(1)
df2.columns = ['percentage']

df2 = df2.reset_index(0).reset_index()
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

df2['Region'] = df2.Parent.map(nameMap)

#df2['phenotype'] = df2.phenotype.map(nMap)

df2

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(4,4))
sns.barplot(data=df2, x='Region', y ='percentage',hue='phenotype',orient='v',hue_order=['Neurons','Oligodendrocytes',
                                                                                        'Astrocytes','Microglia','Other'],
              palette=cMapDict)#['tab:blue','tab:pink'])
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_yticklabels(['0%','10%','20%','30%','40%','50%','60%','70%'])

plt.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/3155_Percentages.png',dpi=300)
#ax.legend_.set_title('')

In [ ]:
adataScaled.obs.ImageType.astype(str)+'-'+ adataScaled.obs.Region.astype(str)

In [ ]:
adataScaled2 = adataScaled.copy()
adataScaled2.obs['Region'] = (adataScaled.obs.ImageType.astype(str)+'-'+ adataScaled.obs.Region.astype(str)).values


In [ ]:
df2 = pd.DataFrame(adataScaled2.obs.groupby('Region').phenotype.value_counts(normalize=True)*100).round(1)
df2.columns = ['percentage']

df2 = df2.reset_index(0).reset_index()
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(4,4))
sns.barplot(data=df2[df2.Region.isin(['AD-GM','Ctl-GM'])], x='Region', y ='percentage',hue='phenotype',orient='v',hue_order=['Neurons','Oligodendrocytes',
                                                                                        'Astrocytes','Microglia','Other'],
              palette=cMapDict)#['tab:blue','tab:pink'])
ax.set_ylabel('% of cells')
ax.set_xlabel('')
ax.set_yticklabels(['0%','10%','20%','30%','40%','50%','60%','70%'])

plt.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/AD_Percentages.png',dpi=300)
#ax.legend_.set_title('')

In [ ]:
df2 = pd.DataFrame(adataScaled2.obs.groupby('Region').phenotype.value_counts(normalize=True)*100).round(1)
df2.columns = ['percentage']

df2 = df2.reset_index(0).reset_index()
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,1,figsize=(4,4))
sns.barplot(data=df2[df2.Region.isin(['AD-WM','Ctl-WM'])], x='Region', y ='percentage',hue='phenotype',orient='v',hue_order=['Neurons','Oligodendrocytes',
                                                                                        'Astrocytes','Microglia','Other'],
              palette=cMapDict)#['tab:blue','tab:pink'])
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_yticklabels(['0%','10%','20%','30%','40%','50%','60%','70%'])

plt.legend().remove()
plt.tight_layout()
plt.savefig('finalFiguresv4/AD_Percentages.png',dpi=300)
#ax.legend_.set_title('')

In [ ]:
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

adataScaled.obs.loc[:,'Region'] = adataScaled.obs.Parent.map(nameMap)
fig,ax=plt.subplots(1,1,figsize=(8,3))
sns.countplot(data=adataScaled[(adataScaled.obs.ImageID=='3155') & (adataScaled.obs.Annotations!='Unknown')].obs, 
              y='Region',hue='Annotations', palette=cMapDict)#['tab:blue','tab:pink'])
ax.set_ylabel('')

In [ ]:
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

adataScaled.obs.loc[:,'Region'] = adataScaled.obs.Parent.map(nameMap)
fig,ax=plt.subplots(2,1,figsize=(8,6))
sns.countplot(data=adataScaled[(adataScaled.obs.ImageID=='3155') & (adataScaled.obs.Annotations!='Unknown')].obs, 
              y='Region',hue='Annotations', hue_order=['Neurons','Astrocytes','Oligodendrocytes','Microglia'],palette=cMapDict, ax=ax[0])#['tab:blue','tab:pink'])
ax[0].set_ylabel('AD')
ax[0].set_xlabel('')

ax[0].get_legend().set_title('')
sns.countplot(data=adataScaled[(adataScaled.obs.ImageID=='3026') & (adataScaled.obs.Annotations!='Unknown')].obs, hue_order=['Neurons','Astrocytes','Oligodendrocytes','Microglia'],
              y='Region',hue='Annotations', palette=cMapDict, ax=ax[1])#['tab:blue','tab:pink'])
ax[1].set_ylabel('Ctl')
ax[1].get_legend().remove()
ax[1].set_xlabel('Cell count')
plt.tight_layout()
plt.savefig('finalFiguresv3/Classification_CellTypes.png', dpi=300)

In [ ]:
nameMap = {'Grey Matter':'GM','White Matter':'WM'}

adataScaled.obs.loc[:,'Region'] = adataScaled.obs.Parent.map(nameMap)
fig,ax=plt.subplots(1,1,figsize=(5,4))
sns.countplot(data=adataScaled[(adataScaled.obs.ImageID=='3026') & (adataScaled.obs.Annotations!='Unknown')].obs, 
              y='Region',hue='Annotations', hue_order=['Neurons','Astrocytes','Oligodendrocytes','Microglia'],palette=cMapDict, ax=ax)#['tab:blue','tab:pink'])
ax.set_ylabel('')
ax.get_legend().remove()
ax.set_xlabel('Cell count')
for p in ax.patches:
    ax.annotate(f'\n{p.get_width()}', (p.get_y()-0.2, p.get_width()), ha='center', va='top', color='k', size=12)
plt.tight_layout()
plt.savefig('finalFiguresv3/Ctl_Classification_CellTypes.png', dpi=300)
